<a href="https://colab.research.google.com/github/yanucci/ronshuku/blob/main/section_2/03_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
今回の演習では、多層パーセプトロン（Multilayer perceptron、MLP）を使いダイヤモンドの価格を予測します。  
MLPのモデルを作成し、ハイパーパラメータをチューニングするコードを記述しましょう。

## PyCaretのインストール

In [18]:
!pip install numpy
!pip install pycaret
!pip install pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


## Google Colabの設定

In [19]:
# from pycaret.utils import enable_colab

# enable_colab()

## データセットの読み込み
「Sarah Gets a Diamond」を読み込みます。  

In [20]:
from pycaret.datasets import get_data

dataset = get_data("diamond")  # データの取得
dataset.shape  # データの形状

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


(6000, 8)

In [21]:
data = dataset.sample(frac=0.9, random_state=786)  # 訓練データ
data_unseen = dataset.drop(data.index)  # 未知のデータ

data.reset_index(drop=True, inplace=True)  # インデックスを初期化
data_unseen.reset_index(drop=True, inplace=True)  # インデックスを初期化

print("訓練データの形状: " + str(data.shape))
print("未知のデータの形状: " + str(data_unseen.shape))

訓練データの形状: (5400, 8)
未知のデータの形状: (600, 8)


## 環境の設定

In [22]:
from pycaret.regression import setup

clf = setup(data=data, target="Price", session_id=123, n_jobs=None)  # 環境の初期化

,Description,Value
0,Session id,123
1,Target,Price
2,Target type,Regression
3,Original data shape,"(5400, 8)"
4,Transformed data shape,"(5400, 29)"
5,Transformed train set shape,"(3779, 29)"
6,Transformed test set shape,"(1621, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


## モデルの作成
以下のコードに追記を行い、「多層パーセプトロン」のモデルを作成しましょう。  

In [24]:
from pycaret.regression import create_model

mlp = create_model("mlp")   # 多層パーセプトロンのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5898.6081,61229295.9393,7824.9151,0.3278,0.6108,0.7075
1,6135.0687,88933198.7701,9430.4400,0.3184,0.6181,0.7030
2,6080.9393,72581570.4728,8519.4818,0.3343,0.6424,0.7603
3,5637.8402,56697466.0915,7529.7720,0.3745,0.5863,0.6574
4,5503.8768,50683056.6211,7119.2034,0.3678,0.6091,0.7228
5,5983.6326,78963197.0016,8886.1238,0.3261,0.6091,0.6989
6,6178.8917,71370677.3937,8448.1168,0.3295,0.6135,0.7034
7,5389.7582,49857295.2655,7060.9699,0.3695,0.5889,0.6723
8,5911.5210,66150397.3946,8133.2895,0.3504,0.5914,0.6658


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
print(mlp)

MLPRegressor(max_iter=500, random_state=123)


## ハイパーパラメータの調整
以下のコードに追記を行い、多層パーセプトロンのハイパーパラメータを調整しましょう。

In [26]:
from pycaret.regression import tune_model

tuned_mlp =  tune_model(mlp)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,690.8149,1224574.1683,1106.6048,0.9866,0.0825,0.0613
1,772.6916,2635440.0838,1623.4039,0.9798,0.0787,0.0601
2,633.9978,1144843.9495,1069.9738,0.9895,0.0777,0.0596
3,646.1626,1238080.8485,1112.6908,0.9863,0.0704,0.0550
4,696.2447,1803350.9423,1342.8890,0.9775,0.0846,0.0636
5,745.8249,2081350.7990,1442.6887,0.9822,0.0787,0.0592
6,703.5782,1590314.6975,1261.0768,0.9851,0.0763,0.0576
7,686.7871,1610148.3512,1268.9162,0.9796,0.0800,0.0602
8,747.5336,2005623.0442,1416.2002,0.9803,0.0810,0.0625


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [37]:
print(tuned_mlp)

MLPRegressor(alpha=0.9, hidden_layer_sizes=[100, 100, 100],
             learning_rate='invscaling', max_iter=500, random_state=123)


## モデルを評価する

In [36]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_mlp)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## デプロイ用のモデルを作成

In [38]:
from pycaret.regression import finalize_model

final_mlp = finalize_model(tuned_mlp)
print(final_mlp)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Carat Weight'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                             'Polish', 'Symmetry', 'Report'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['Rep...
                                             'Polish', 'Symmetry'],
                                    transformer=OneHotEncoder(cols=['Cut',
                                                                    'Color',
                                                                    'Clarity',
                                                                    'Polish',
                                              

In [39]:
from pycaret.regression import predict_model

predict_model(final_mlp);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,MLP Regressor,620.5763,1164320.2386,1079.0367,0.9894,0.0725,0.0573


In [40]:
from pycaret.regression import save_model

save_model(final_mlp, "mlp_model")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Carat Weight'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                              'Polish', 'Symmetry', 'Report'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['Rep...
                                              'Polish', 'Symmetry'],
                                     transformer=OneHotEncoder(cols=['Cut',
                                                                     'Color',
                                                                     'Clarity',
                                                                     'Polish',
                               

In [41]:
from pycaret.regression import load_model

loaded_lr = load_model("mlp_model")

Transformation Pipeline and Model Successfully Loaded


## 未知のデータで予測する

In [42]:
unseen_pred = predict_model(loaded_lr, data=data_unseen)
unseen_pred.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,MLP Regressor,757.8983,1806698.4505,1344.1348,0.9824,0.0800,0.0635


,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,prediction_label
0,1.53,Ideal,E,SI1,ID,ID,AGSL,12791,12578.554503
1,1.50,Fair,F,SI1,VG,VG,GIA,10450,10327.656388
2,1.01,Good,E,SI1,G,G,GIA,5161,5143.370741
3,2.51,Very Good,G,VS2,VG,VG,GIA,34361,34808.758529
4,1.01,Good,I,SI1,VG,VG,GIA,4238,4128.368597


## 解答例
以下は解答例です。

### モデルの作成

In [34]:
from pycaret.regression import create_model

mlp = create_model("mlp")  # 多層パーセプトロンのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5898.6081,61229295.9393,7824.9151,0.3278,0.6108,0.7075
1,6135.0687,88933198.7701,9430.4400,0.3184,0.6181,0.7030
2,6080.9393,72581570.4728,8519.4818,0.3343,0.6424,0.7603
3,5637.8402,56697466.0915,7529.7720,0.3745,0.5863,0.6574
4,5503.8768,50683056.6211,7119.2034,0.3678,0.6091,0.7228
5,5983.6326,78963197.0016,8886.1238,0.3261,0.6091,0.6989
6,6178.8917,71370677.3937,8448.1168,0.3295,0.6135,0.7034
7,5389.7582,49857295.2655,7060.9699,0.3695,0.5889,0.6723
8,5911.5210,66150397.3946,8133.2895,0.3504,0.5914,0.6658


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### ハイパーパラメータの調整

In [35]:
from pycaret.regression import tune_model

tuned_mlp = tune_model(mlp)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,690.8149,1224574.1683,1106.6048,0.9866,0.0825,0.0613
1,772.6916,2635440.0838,1623.4039,0.9798,0.0787,0.0601
2,633.9978,1144843.9495,1069.9738,0.9895,0.0777,0.0596
3,646.1626,1238080.8485,1112.6908,0.9863,0.0704,0.0550
4,696.2447,1803350.9423,1342.8890,0.9775,0.0846,0.0636
5,745.8249,2081350.7990,1442.6887,0.9822,0.0787,0.0592
6,703.5782,1590314.6975,1261.0768,0.9851,0.0763,0.0576
7,686.7871,1610148.3512,1268.9162,0.9796,0.0800,0.0602
8,747.5336,2005623.0442,1416.2002,0.9803,0.0810,0.0625


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
